# Modèle binomial de Cox–Ross–Rubinstein (CRR)

Le modèle de Cox–Ross–Rubinstein est un modèle discret d’évaluation des options fondé sur la construction d’un arbre binomial représentant l’évolution possible du prix d’un actif sous-jacent.  
Il repose sur l’idée qu’à chaque pas de temps, le prix de l’actif peut soit augmenter d’un facteur déterminé, soit diminuer d’un autre facteur déterminé.  

### Dynamique de l’actif

On considère un actif sous-jacent de prix $S_t$ au temps $t$.  
L’échéance de l’option est $T$, et l’on divise l’intervalle $[t, T]$ en $N$ sous-intervalles de longueur :

$$
\Delta t = \frac{T - t}{N}
$$

Le temps restant jusqu’à l’échéance est :

$$
\tau = T - t
$$

À chaque pas de temps, le sous-jacent peut :

augmenter d’un facteur $u$ (hausse) <br>
diminuer d’un facteur $d$ (baisse)

Ces facteurs sont choisis de façon à reproduire la volatilité $\sigma$ du sous-jacent :

$$
u = e^{\sigma \sqrt{\Delta t}}, \quad d = e^{-\sigma \sqrt{\Delta t}} = \frac{1}{u}
$$

Sous la mesure risque-neutre $\mathbb{Q}$, la probabilité d’une hausse est donnée par :

$$
p = \frac{e^{r \Delta t} - d}{u - d}
$$

où $r$ est le taux d’intérêt sans risque.  
On vérifie que $0 < p < 1$ si $d < e^{r \Delta t} < u$.

### Valeur du sous-jacent à maturité

Après $M = N - i$ étapes restantes depuis la date $t$, si le sous-jacent connaît $j$ baisses (et donc $M - j$ hausses), sa valeur est :

$$
S_T = S_t\, u^{\,M - j} d^{\,j}
$$

où $S_t$ est le prix du sous-jacent au temps $t$.

### Prix d'une option européenne dans le modèle CRR

La valeur d'une option européenne au temps $t$ est l'espérance actualisée de son payoff sous la probabilité risque-neutre.  

 Pour un call européen :

$$
C_t = e^{-r \tau} \sum_{j=0}^{M} \binom{M}{j} p^{\,M - j} (1 - p)^{j} \max\left( S_t\, u^{\,M - j} d^{\,j} - K,\, 0 \right)
$$
<br><br>
 Pour un put européen:

$$
P_t = e^{-r \tau} \sum_{j=0}^{M} \binom{M}{j} p^{\,M - j} (1 - p)^{j} \max\left( K - S_t\, u^{\,M - j} d^{\,j},\, 0 \right)
$$

avec :
<br>
$M = N - i$ : nombre de pas restants à partir du temps $t$  <br>
$\tau = T - t$ : temps restant jusqu'à l’échéance  <br>
$p = \dfrac{e^{r \Delta t} - d}{u - d}$ : probabilité risque-neutre d’une hausse <br>  
$\binom{M}{j} = \dfrac{M!}{j! (M - j)!}$ : coefficient binomial  <br>

### Interprétation

Chaque terme de la somme représente le payoff pondéré par la probabilité d’un scénario donné :  <br>
 $j$ baisses et $M - j$ hausses sur les $M$ pas restants,  <br>
$S_t\, u^{M - j} d^{j}$ est le prix atteint dans ce scénario,  le coefficient binomial $\binom{M}{j}$ représente le nombre de chemins possibles correspondant à ce scénario. <br>

L’actualisation $e^{-r \tau}$ permet de ramener cette espérance à la date $t$.

Rq: Plus $N$ est grand (pas de temps petits), plus le modèle CRR converge vers le modèle continu de Black–Scholes.


In [18]:
import numpy as np

class BinomialModel:
    def __init__(self, S0=100, K=100, T=1.0, N=3, u=1.1, r=0.06, optype="C"):
        self.S0 = float(S0)
        self.K = float(K)
        self.T = float(T)
        self.N = int(N)
        self.r = float(r)
        self.optype = optype.upper()

        self.dt = self.T / self.N
        self.u = float(u)
        self.d = 1 / self.u

        if not (self.d < np.exp(self.r * self.dt) < self.u):
            raise ValueError("Condition d'absence d'arbitrage non respectée.")

    def _risk_neutral_proba(self):
        q = (np.exp(self.r * self.dt) - self.d) / (self.u - self.d)
        disc = np.exp(-self.r * self.dt)
        return q, disc

    def option_price_time_t(self, S_t=None, t_index=0):

        if S_t is None:
            S_t = self.S0
        if not (0 <= t_index <= self.N):
            raise ValueError("t_index doit être compris entre 0 et N.")

        q, disc = self._risk_neutral_proba()
        M = self.N - t_index  # nombre d’étapes restantes
        j = np.arange(M + 1)
        S_T = S_t * (self.u ** j) * (self.d ** (M - j))

        if self.optype == "C":
            C = np.maximum(S_T - self.K, 0.0)
        else:
            C = np.maximum(self.K - S_T, 0.0)

        # backward sur les étapes restantes
        for i in range(M, 0, -1):
            C = disc * (q * C[1:i + 1] + (1 - q) * C[0:i])

        return C[0]


## Exemple

Prix initial put

In [30]:
put = BinomialModel(S0=100, K=100, T=1, N=3, u=1.1, r=0.05, optype="P")
print("Prix put à t=0 :", put.option_price_time_t())


Prix put à t=0 : 4.73727527375481


Prix initial Call

In [34]:
call = BinomialModel(S0=100, K=100, T=1, N=3, u=1.1, r=0.05, optype="C")
print("Prix à t=0 :", call.option_price_time_t())

Prix à t=0 : 9.614332823683386


Prix call après deux hausses

In [39]:
call2 = call.S0 * call.u**2
print("Prix call à t=2 après deux hausses :", call.option_price_time_t(S_t=S2, t_index=2))


Prix call à t=2 après deux hausses : 22.65285461783826


Prix put après deux hausses 

In [42]:
put2 = put.S0 * put.u**2
print("Prix put à t=2 après deux hausses :", put.option_price_time_t(S_t=S2, t_index=2))


Prix put à t=2 après deux hausses : 0.0
